# BITCOIN PRICE FORCASTING USING VAR,VARMA AND VARMA with AUTO-ARIMA


# **INTRODUCTION**

In this project, we are going to choose the best model beteween the following multivariate time series models : VAR,VARMA AND VARMA with AUTO-ARIMA to forcast bitcoin price.

Why MTS?

 Multivariate Time Series consist of more than one time-dependent variable and each variable depends not only on its past values but also has some dependency on other variables.

Bitcoin’s movements show now a high correlation with those of Nasdaq index and S&P500. That's why we will validate if there is a real causality relation between them so we can use multivariate time series models for forecasting.


  



# **Import Necessary Libraries**

We should get the data we will work on, so let's instal **yfinance** library to get bitcoin dataset from 2017-03-21 to 2022-03-21:

In [ ]:
pip install yfinance

In [ ]:
!pip install statsmodels==0.11 #this version is necessary to work with pmdarima library

In [ ]:
pip install pmdarima

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
import statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

import sklearn
from sklearn.model_selection import ParameterGrid
from sklearn import metrics
from sklearn.model_selection import ParameterGrid

from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX
from pmdarima import auto_arima

import warnings
warnings.filterwarnings("ignore")
from pmdarima.arima import auto_arima


# **Dataset Construction**

### **A. BTC Dataset** 

In [5]:
data= yf.download("BTC-USD", start="2017-03-21", end="2022-03-21")
data=data.rename(columns={"High": "BTC High",
                   "Volume": "BTC Volume"})
data.drop(['Open','Low','Close','Adj Close','BTC Volume'], axis='columns', inplace=True) #We will work with high value only
data = data.reset_index(level=0) # to make the date a variable
data['Date'] = data['Date'].dt.strftime('%m/%d/%Y') # to make the date format like in the next dataset
data

[*********************100%***********************]  1 of 1 completed


,Date,BTC High
0,03/21/2017,1122.430054
1,03/22/2017,1120.650024
2,03/23/2017,1058.010010
3,03/24/2017,1040.469971
4,03/25/2017,975.760986
...,...,...
1822,03/17/2022,41287.535156
1823,03/18/2022,42195.746094
1824,03/19/2022,42316.554688
1825,03/20/2022,42241.164062


### **B. S&P500 and NASDAQ Dataset** 

To get the SP500&NASDAQ datasets, we used the one given by Yahoo Finance.
You can download it from [SP500 dataset](https://finance.yahoo.com/quote/%5EGSPC/history/) and [NASDAQ dataset](https://finance.yahoo.com/quote/%5EIXIC/history?period1=1490054400&period2=1647820800&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true).

In [6]:
from google.colab import files 
import io 
uploaded = files.upload()

Saving ^IXIC.csv to ^IXIC.csv
Saving SP500.csv to SP500.csv


In [8]:
df=pd.read_csv('SP500.csv')
dfn=pd.read_csv('^IXIC.csv')
dfn['Date'] = pd.to_datetime(dfn['Date'])
dfn['Date'] = dfn['Date'].dt.strftime('%m/%d/%Y') # to make the date format like the others
dfn

,Date,Open,High,Low,Close,Adj Close,Volume
0,03/21/2017,5923.419922,5928.060059,5790.729980,5793.830078,5793.830078,2192090000
1,03/22/2017,5790.589844,5825.669922,5781.799805,5821.640137,5821.640137,1857680000
2,03/23/2017,5812.310059,5842.819824,5806.979980,5817.689941,5817.689941,1742640000
3,03/24/2017,5839.330078,5858.950195,5807.830078,5828.740234,5828.740234,1841920000
4,03/27/2017,5776.330078,5849.200195,5769.390137,5840.370117,5840.370117,1670760000
...,...,...,...,...,...,...,...
1254,03/14/2022,12795.120117,12918.009766,12555.349609,12581.219727,12581.219727,5853360000
1255,03/15/2022,12685.230469,12973.879883,12616.589844,12948.620117,12948.620117,5414590000
1256,03/16/2022,13119.370117,13440.120117,12992.200195,13436.549805,13436.549805,6498110000
1257,03/17/2022,13360.719727,13620.799805,13317.139648,13614.780273,13614.780273,5575030000


 >SP500 and NASDAQ dataset cleaning

In [9]:
df=df.rename(columns={"High": "SP500 High"})
df.drop(['Open','Low','Close','Unnamed: 0'], axis='columns', inplace=True) # Since we are working only with high value
df = df.iloc[::-1] # to sort the SP500 dataset the same way as BTC dataset 
dfn=dfn.rename(columns={"High": "NASDAQ High"})
dfn.drop(['Open','Low','Close','Adj Close','Volume'], axis='columns', inplace=True) # Since we are working only with high value
dfn

,Date,NASDAQ High
0,03/21/2017,5928.060059
1,03/22/2017,5825.669922
2,03/23/2017,5842.819824
3,03/24/2017,5858.950195
4,03/27/2017,5849.200195
...,...,...
1254,03/14/2022,12918.009766
1255,03/15/2022,12973.879883
1256,03/16/2022,13440.120117
1257,03/17/2022,13620.799805


### **C. Merge S&P500&NASDAQ&BTC Datasets**

In [10]:
result = pd.merge(data, df, how='outer')
result = pd.merge(result, dfn, how='outer')
result

,Date,BTC High,SP500 High,NASDAQ High
0,03/21/2017,1122.430054,"2,381.93",5928.060059
1,03/22/2017,1120.650024,"2,351.81",5825.669922
2,03/23/2017,1058.010010,"2,358.92",5842.819824
3,03/24/2017,1040.469971,"2,356.22",5858.950195
4,03/25/2017,975.760986,NaN,NaN
...,...,...,...,...
1822,03/17/2022,41287.535156,"4,412.67",13620.799805
1823,03/18/2022,42195.746094,"4,465.40",13899.280273
1824,03/19/2022,42316.554688,NaN,NaN
1825,03/20/2022,42241.164062,NaN,NaN


> To fill the missing values of S&P500 and NASDAQ columns, we will dynamically replace them with the average values of previous and next non-missing values so we don't lose much information.

In [11]:
for i in range (1,1823):
  if pd.isna(result['SP500 High'].iloc[i]):#Check if the value is null
    j=i+1
    while pd.isna(result['SP500 High'].iloc[j]): #search the next non-null value
      j=j+1
    prev=result['SP500 High'].iloc[i-1]
    prev=float(prev.replace(',',''))
    next=result['SP500 High'].iloc[j]
    next=float(next.replace(',',''))
    result['SP500 High'].iloc[i]=str((prev+next)/2) 
for i in range (1,1823):
  if pd.isna(result['NASDAQ High'].iloc[i]):#Check if the value is null
    j=i+1
    while pd.isna(result['NASDAQ High'].iloc[j]): #search the next non-null value
      j=j+1
    prev=result['NASDAQ High'].iloc[i-1]
    next=result['NASDAQ High'].iloc[j]
    result['NASDAQ High'].iloc[i]=float((prev+next)/2) 
data = result.dropna()

for i in range (1824):
  a=data['SP500 High'].iloc[i]
  data['SP500 High'].iloc[i]=float(a.replace(',',''))

data

,Date,BTC High,SP500 High,NASDAQ High
0,03/21/2017,1122.430054,2381.93,5928.060059
1,03/22/2017,1120.650024,2351.81,5825.669922
2,03/23/2017,1058.010010,2358.92,5842.819824
3,03/24/2017,1040.469971,2356.22,5858.950195
4,03/25/2017,975.760986,2350.56,5854.075195
...,...,...,...,...
1819,03/14/2022,39742.500000,4247.57,12918.009766
1820,03/15/2022,39794.628906,4271.05,12973.879883
1821,03/16/2022,41465.453125,4358.9,13440.120117
1822,03/17/2022,41287.535156,4412.67,13620.799805


## **Data Exploration**

### **1. Visualize the Time Series**

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Date'], y=data['BTC High'],
                    mode='lines',
                    name='BTC High',
                    line=dict(color='#a5ade6')))

fig.add_trace(go.Scatter(x=data['Date'], y=data['SP500 High'],
                    mode='lines', name='SP500 High',
                    line=dict(color='#2eb0c7')))

fig.add_trace(go.Scatter(x=data['Date'], y=data['NASDAQ High'],
                    mode='lines', name='NASDAQ High',
                    line=dict(color='#E5EA35')))
fig.update_layout(font_color="#d7dbf5",
                  paper_bgcolor="#020938",  
                  plot_bgcolor="#020938",title="Visualize BTC, NASDAQ and S&P500 evolution")
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        ),
    ),
fig.show()

# **Building models**

The data is ready, let’s start the trip of MTS modeling! We will involve the steps below:
*   Causality investigation.
*    Test for stationary.
*   Model Building.






### **Split data into Training and Testing Data**

select 80% of data as the training data and the rest as the test data

In [13]:
data=data.set_index('Date')

In [14]:
data["SP500 High"] = pd.to_numeric(data["SP500 High"])
data["NASDAQ High"] = pd.to_numeric(data["NASDAQ High"])

In [15]:
data_train, data_test = data[0:-90], data[-90:] #The last three months will be used to test the models
data_test

,BTC High,SP500 High,NASDAQ High
Date,,,
12/19/2021,48089.664062,4607.60,15077.669922
12/20/2021,47401.718750,4587.90,15007.299805
12/21/2021,49300.917969,4651.14,15349.059570
12/22/2021,49544.796875,4697.67,15525.969727
12/23/2021,51332.339844,4740.74,15697.980469
...,...,...,...
03/14/2022,39742.500000,4247.57,12918.009766
03/15/2022,39794.628906,4271.05,12973.879883
03/16/2022,41465.453125,4358.90,13440.120117


### **Testing Causation using Granger’s Causality Test**

First, we use Granger Causality Test to investigate causality of data. Granger causality is a way to investigate the causality between two variables in a time series which actually means if a particular variable comes before another in the time series. In the MTS, we will test the causality of all combinations of pairs of variables.

The Null Hypothesis of the Granger Causality Test is that lagged column-values do not explain the variation in index, so the column does not cause index. We use grangercausalitytests function in the package statsmodels to do the test and the output of the matrix is the minimum p-value when computes the test for all lags up to maxlag. The critical value we use is 5% and if the p-value of a pair of variables is smaller than 0.05, we could say with 95% confidence that a predictor colomn causes a response index .

In [16]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test' #since we are working with multivariate time series, we should use Wald statistic that asymptotically follows a chi-square distribution
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var  for var in variables]
    df.index = [var for var in variables]
    return df
variables=['BTC High','SP500 High','NASDAQ High']
grangers_causation_matrix(data, variables) 

,BTC High,SP500 High,NASDAQ High
BTC High,1.0000,0.0185,0.0002
SP500 High,0.0339,1.0000,0.0016
NASDAQ High,0.1810,0.0012,1.0000


P-Values lesser than the significance level (0.05), implies the Null Hypothesis (the X does not cause Y) can be rejected.


Looking at the P-Values in the table, we can see that S&P500 and NASDAQ series are causing The Bitcoin serie. This makes this system of time series a good candidate for using MTS to predict future values.






### **Check stationarity and stationarize Time Series**

As VAR ,VARIMA and VARIMA with auto_ARIMA requires time series to be stationary, we will use one popular statistical test – Augmented Dickey-Fuller Test (ADF Test) to check the stationary of each variable in the dataset. If the stationarity is not achieved, we need to make the data stationary, such as eliminating the trend and seasonality by differencing and seasonal decomposition.

In the following script, we use adfuller function in the statsmodels package for stationary test of each variables. The Null Hypothesis is that the data has unit root and is not stationary and the significant value is 0.05.

In [17]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*44)
    print(f' Null Hypothesis: Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Time series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Accepting Null Hypothesis.")
        print(f" => Time series is Non-Stationary.")

In [18]:
for name, column in data_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "BTC High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -0.6822
 No. Lags Chosen       = 22
 => P-Value = 0.8513. Accepting Null Hypothesis.
 => Time series is Non-Stationary.


    Augmented Dickey-Fuller Test on "SP500 High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 0.0508
 No. Lags Chosen       = 13
 => P-Value = 0.9625. Accepting Null Hypothesis.
 => Time series is Non-Stationary.


    Augmented Dickey-Fuller Test on "NASDAQ High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 0.2985
 No. Lags Chosen       = 1
 => P-Value = 0.9773. Accepting Null Hypothesis.
 => Time series is Non-Stationary.




The ADF test shows that our time series are not stationary. Let’s see what about their first difference.

In [19]:
data_differenced = data_train.diff().dropna()
for name, column in data_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "BTC High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -7.3628
 No. Lags Chosen       = 25
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Time series is Stationary.


    Augmented Dickey-Fuller Test on "SP500 High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -9.9999
 No. Lags Chosen       = 12
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Time series is Stationary.


    Augmented Dickey-Fuller Test on "NASDAQ High" 
    --------------------------------------------
 Null Hypothesis: Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -35.1583
 No. Lags Chosen       = 0
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Time series is Stationary.




Now, we have stationarized all our time series.

In [20]:
data_differenced

,BTC High,SP500 High,NASDAQ High
Date,,,
03/22/2017,-1.780029,-30.12,-102.390137
03/23/2017,-62.640015,7.11,17.149902
03/24/2017,-17.540039,-2.70,16.130371
03/25/2017,-64.708984,-5.66,-4.875000
03/26/2017,32.199036,-2.83,-2.437500
...,...,...,...
12/14/2021,-1773.601562,-49.83,-319.509765
12/15/2021,1042.558594,52.13,258.209961
12/16/2021,-48.382812,19.39,57.430664


# **VAR model**

### **1. Select the Order of VAR model**

In [21]:
model = VAR(data_differenced)
x = model.select_order(maxlags=12)
x.summary()

,AIC,BIC,FPE,HQIC
0,26.47,26.47,3.118e+11,26.47
1,26.39,26.43*,2.890e+11,26.40*
2,26.39,26.46,2.890e+11,26.41
3,26.39,26.49,2.896e+11,26.43
4,26.38*,26.50,2.854e+11*,26.42
5,26.38,26.53,2.865e+11,26.44
6,26.39,26.57,2.881e+11,26.45
7,26.39,26.60,2.903e+11,26.47
8,26.39,26.63,2.900e+11,26.48
9,26.39,26.66,2.902e+11,26.49


### **2. Training**

In [22]:
model_fitted = model.fit(4)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 14, May, 2022
Time:                     17:57:51
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    26.4874
Nobs:                     1729.00    HQIC:                   26.4099
Log likelihood:          -30113.0    FPE:                2.81768e+11
AIC:                      26.3644    Det(Omega_mle):     2.75507e+11
--------------------------------------------------------------------
Results for equation BTC High
                    coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------
const                 16.420412        19.310936            0.850           0.395
L1.BTC High            0.119635         0.024081            4.968           0.000
L1.SP500 High         -4.272842         2.115597     

### **3. Model Forecasting**

In [23]:
lag_order = model_fitted.k_ar
lag_order

4

In [24]:
forecast_input = data_differenced.values[-lag_order:]
forecast_input

array([[ 1042.55859375,    52.13      ,   258.209961  ],
       [  -48.3828125 ,    19.39      ,    57.430664  ],
       [-1420.6796875 ,   -65.29      ,  -344.410157  ],
       [ -691.06640625,   -39.4       ,  -140.740234  ]])

In [31]:
fc = model_fitted.forecast(y=forecast_input, steps=90)
df_forecast = pd.DataFrame(fc, index=data.index[-90:], columns=data.columns + '_1d')
df_forecast

,BTC High_1d,SP500 High_1d,NASDAQ High_1d
Date,,,
12/19/2021,188.989455,-9.048129,-35.350514
12/20/2021,1.681573,-6.328173,-12.276871
12/21/2021,-179.083629,-0.158463,11.335535
12/22/2021,-126.414237,0.551770,5.274261
12/23/2021,6.817410,1.671658,7.250271
...,...,...,...
03/14/2022,26.703297,1.306036,5.347191
03/15/2022,26.703297,1.306036,5.347191
03/16/2022,26.703297,1.306036,5.347191


In [32]:
def invert_transformation(df_train, df_forecast):
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:
      df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [33]:
df_results = invert_transformation(data_train, df_forecast)
df_results

,BTC High_1d,SP500 High_1d,NASDAQ High_1d,BTC High_forecast,SP500 High_forecast,NASDAQ High_forecast
Date,,,,,,
12/19/2021,188.989455,-9.048129,-35.350514,47502.817580,4618.251871,15112.689525
12/20/2021,1.681573,-6.328173,-12.276871,47504.499153,4611.923698,15100.412654
12/21/2021,-179.083629,-0.158463,11.335535,47325.415524,4611.765235,15111.748188
12/22/2021,-126.414237,0.551770,5.274261,47199.001287,4612.317005,15117.022449
12/23/2021,6.817410,1.671658,7.250271,47205.818697,4613.988663,15124.272721
...,...,...,...,...,...,...
03/14/2022,26.703297,1.306036,5.347191,49332.671826,4720.410483,15560.198600
03/15/2022,26.703297,1.306036,5.347191,49359.375123,4721.716519,15565.545792
03/16/2022,26.703297,1.306036,5.347191,49386.078420,4723.022556,15570.892983


### **4. Visualize forcast VS actual**








In [34]:
fig = go.Figure()
fig1 = go.Figure()
fig2 = go.Figure()
fig.add_trace(go.Scatter(x=data_test.index, y=data_test['BTC High'],
                    mode='lines',
                    name='BTC High',
                    line=dict(color='#a03fba')))

fig1.add_trace(go.Scatter(x=data_test.index, y=data_test['NASDAQ High'],
                    mode='lines',
                    name='NASDAQ High',
                    line=dict(color='#a03fba')))
fig2.add_trace(go.Scatter(x=data_test.index, y=data_test['SP500 High'],
                    mode='lines', name='SP500 High',
                    line=dict(color='#a03fba')))
fig.add_trace(go.Scatter(x=data_test.index, y=df_results['BTC High_forecast'],
                    mode='lines',
                    name='BTC High forecast',
                    line=dict(color='#2eb0c7')))

fig1.add_trace(go.Scatter(x=data_test.index, y=df_results['NASDAQ High_forecast'],
                    mode='lines',
                    name='NASDAQ High',
                    line=dict(color='#2eb0c7')))
fig2.add_trace(go.Scatter(x=data_test.index, y=df_results['SP500 High_forecast'],
                    mode='lines', name='SP500 High forecast',
                    line=dict(color='#2eb0c7')))
for plot in [fig,fig1,fig2]:
  plot.update_layout(font_color="#d7dbf5",
                  paper_bgcolor="#020938",  
                  plot_bgcolor="#020938",title="Forcast VS Actual")
  plot.update_layout(
      xaxis=dict(
          showline=True,
          showgrid=False,
          showticklabels=True,
          ),
      )

### **5. Forecast Evaluation For VAR model**

In [35]:
from statsmodels.tsa.stattools import acf
def adjust(val, length= 6): return str(val).ljust(length)
def forecast_accuracy(forecast, actual):
    m=np.abs(forecast - actual)/np.abs(actual)
    mape = np.mean(m)  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE

    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse})

print('Forecast Accuracy of: BTC HIGH')
accuracy_prod = forecast_accuracy(df_results['BTC High_forecast'].values, data_test['BTC High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))


print('\nForecast Accuracy of: SP500')
accuracy_prod = forecast_accuracy(df_results['SP500 High_forecast'].values, data_test['SP500 High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('\nForecast Accuracy of: NASDAQ')
accuracy_prod = forecast_accuracy(df_results['NASDAQ High_forecast'].values, data_test['SP500 High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

Forecast Accuracy of: BTC HIGH
mape   :  0.1496
me     :  5382.6403
mae    :  6112.3827
mpe    :  0.1353
rmse   :  6904.3836

Forecast Accuracy of: SP500
mape   :  0.0467
me     :  138.9995
mae    :  207.4629
mpe    :  0.0324
rmse   :  242.8535

Forecast Accuracy of: NASDAQ
mape   :  2.3938
me     :  10814.8386
mae    :  10814.8386
mpe    :  2.3938
rmse   :  10818.9816


# **VARMA model**


### **1. Select the Order of** **VARMA model**




In [36]:
param_grid = {'p': [1,2,3], 'q':[1,2,3], 'tr': ['n','c','t','ct']}
pg = list(ParameterGrid(param_grid))

In [37]:
def inverse_diff(actual_df, pred_df):
    '''
    Transforms the differentiated values back
    
    Args:
        actual dataframe (float64): Values of the columns, numpy array of floats 
        predicted dataframe (float64): Values of the columns, numpy array of floats 
    
    Returns:
        Dataframe with the predicted values
    '''
    df_res = pred_df.copy()
    columns = actual_df.columns
    for col in columns: 
        df_res[str(col)+'_1st_inv_diff'] = actual_df[col].iloc[-1] + df_res[str(col)].cumsum()
    return df_res

In [38]:
df_results_VARMA = pd.DataFrame(columns=['p', 'q', 'tr','RMSE btchigh','RMSE sphigh','RMSE nashigh'])

for a,b in enumerate(pg):
    p = b.get('p')
    q = b.get('q')
    tr = b.get('tr')
    model = VARMAX(data_differenced, order=(p,q), trend=tr)
    model_fit=model.fit()
    z = model_fit.forecast(y=data_differenced[['BTC High', 'SP500 High','NASDAQ High']],steps=len(data_test))
    df_pred = pd.DataFrame(z, columns=['BTC High', 'SP500 High','NASDAQ High'])
    res = inverse_diff(data[['BTC High', 'SP500 High','NASDAQ High']],df_pred)
    highrmse = np.sqrt(metrics.mean_squared_error(data_test['BTC High'], res['BTC High_1st_inv_diff']))
    sphighrmse = np.sqrt(metrics.mean_squared_error(data_test['SP500 High'], res['SP500 High_1st_inv_diff']))
    nashighrmse = np.sqrt(metrics.mean_squared_error(data_test['NASDAQ High'], res['NASDAQ High_1st_inv_diff']))
    df_results_VARMA = df_results_VARMA.append({'p': p, 'q': q, 'tr': tr,'RMSE btchigh':highrmse,'RMSE sphigh':sphighrmse,'RMSE nashigh':nashighrmse}, ignore_index=True)

In [39]:
df_results_VARMA.sort_values(by=['RMSE btchigh','RMSE sphigh','RMSE nashigh']).head()


,p,q,tr,RMSE btchigh,RMSE sphigh,RMSE nashigh
8,1,3,n,3952.836738,186.674728,1001.786115
20,2,3,n,3954.135079,187.111819,1003.078006
16,2,2,n,3954.455653,185.925001,1001.955756
12,2,1,n,3957.830223,186.133186,1002.255106
32,3,3,n,3959.657662,187.514633,1004.293599


In [40]:
best_values_VARMA = df_results_VARMA.sort_values(by=['RMSE btchigh','RMSE sphigh']).head(1)
best_values_VARMA

,p,q,tr,RMSE btchigh,RMSE sphigh,RMSE nashigh
8,1,3,n,3952.836738,186.674728,1001.786115


Finaly , we got the best order of VARMA model 

In [41]:
p_value_VARMA = best_values_VARMA['p'].iloc[0]
q_value_VARMA = best_values_VARMA['q'].iloc[0] 
tr_value_VARMA = best_values_VARMA['tr'].iloc[0] 

print("p_value_VARMA: ", p_value_VARMA)
print("q_value_VARMA: ", q_value_VARMA)
print("tr_value_VARMA: ", tr_value_VARMA)

p_value_VARMA:  1
q_value_VARMA:  3
tr_value_VARMA:  n


### **2. Forcasting of VARMA model** 

In this section, we will use forcast() function of VARMA to get the forecast 
results and then evaluate the forecasts with data_test.

In [42]:
model_fit = VARMAX(data_differenced[['BTC High', 'SP500 High','NASDAQ High']], 
               order=(p_value_VARMA, q_value_VARMA),trends = tr_value_VARMA).fit(disp=False)
result = model_fit.forecast(steps = len(data_test))

In [43]:
res = inverse_diff(data[['BTC High', 'SP500 High','NASDAQ High']], result)
res.head()

,BTC High,SP500 High,NASDAQ High,BTC High_1st_inv_diff,SP500 High_1st_inv_diff,NASDAQ High_1st_inv_diff
2021-12-19,74.808330,-8.714678,-31.565569,42270.554424,4456.685322,13867.714704
2021-12-20,-34.756739,-5.213573,-9.962041,42235.797685,4451.471750,13857.752662
2021-12-21,22.103244,0.562355,6.506965,42257.900929,4452.034104,13864.259627
2021-12-22,27.895928,1.073819,5.295072,42285.796858,4453.107924,13869.554699
2021-12-23,25.983100,1.226203,5.246419,42311.779958,4454.334127,13874.801118


In [44]:
res["new_index"] = range(len(data_train), len(data))
res =res.set_index("new_index")
res

,BTC High,SP500 High,NASDAQ High,BTC High_1st_inv_diff,SP500 High_1st_inv_diff,NASDAQ High_1st_inv_diff
new_index,,,,,,
1734,74.808330,-8.714678,-31.565569,42270.554424,4456.685322,13867.714704
1735,-34.756739,-5.213573,-9.962041,42235.797685,4451.471750,13857.752662
1736,22.103244,0.562355,6.506965,42257.900929,4452.034104,13864.259627
1737,27.895928,1.073819,5.295072,42285.796858,4453.107924,13869.554699
1738,25.983100,1.226203,5.246419,42311.779958,4454.334127,13874.801118
...,...,...,...,...,...,...
1819,25.350453,1.277151,5.285064,44365.295233,4557.768303,14302.871631
1820,25.350453,1.277151,5.285064,44390.645686,4559.045454,14308.156695
1821,25.350453,1.277151,5.285064,44415.996139,4560.322606,14313.441760


### **3. Visualize forcast VS actual**

Visualize the forecast with actual values:


In [45]:
fig = go.Figure()
fig3 = go.Figure()
fig2 = go.Figure()

fig.add_trace(go.Scatter(x=data_test.index, y=data_test['BTC High'],
                    mode='lines',
                    name='BTC High',
                    line=dict(color='#a03fba')))

fig2.add_trace(go.Scatter(x=data_test.index, y=data_test['SP500 High'],
                    mode='lines', name='SP500 High',
                    line=dict(color='#a03fba')))
fig3.add_trace(go.Scatter(x=data_test.index, y=data_test['NASDAQ High'],
                    mode='lines', name='NASDAQ High',
                    line=dict(color='#a03fba')))
fig.add_trace(go.Scatter(x=data_test.index, y=res['BTC High_1st_inv_diff'],
                    mode='lines',
                    name='BTC High forecast',
                    line=dict(color='#2eb0c7')))


fig2.add_trace(go.Scatter(x=data_test.index, y=res['SP500 High_1st_inv_diff'],
                    mode='lines', name='SP500 High forecast',
                    line=dict(color='#2eb0c7')))

fig3.add_trace(go.Scatter(x=data_test.index, y=res['NASDAQ High_1st_inv_diff'],
                    mode='lines', name='NASDAQ High forecast',
                    line=dict(color='#2eb0c7')))
for plot in [fig,fig2,fig3]:
  plot.update_layout(font_color="#d7dbf5",
                  paper_bgcolor="#020938",  
                  plot_bgcolor="#020938",title="Forcast VS Actual")
  plot.update_layout(
      xaxis=dict(
          showline=True,
          showgrid=False,
          showticklabels=True,
          ),
      )
  plot.show()

### **4. Forecast Evaluation**

Then, use forcast_accuracy() function of hana-ml to evaluate the forecasts with metric ‘rmse’,'MAPE','MAE','ME','MPE'.


In [46]:
from statsmodels.tsa.stattools import acf
def adjust(val, length= 6): return str(val).ljust(length)
def forecast_accuracy(forecast, actual):
    m=np.abs(forecast - actual)/np.abs(actual)
    mape = np.mean(m)  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE

    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse})

print('Forecast Accuracy of: BTC HIGH')
accuracy_prod = forecast_accuracy(res['BTC High_1st_inv_diff'].values, data_test['BTC High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))


print('\nForecast Accuracy of: SP500')
accuracy_prod = forecast_accuracy(res['SP500 High_1st_inv_diff'].values, data_test['SP500 High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))
    
print('\nForecast Accuracy of: NASDAQ')
accuracy_prod = forecast_accuracy(res['NASDAQ High_1st_inv_diff'].values, data_test['NASDAQ High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

Forecast Accuracy of: BTC HIGH
mape   :  0.0823
me     :  456.9165
mae    :  3524.3016
mpe    :  0.0195
rmse   :  4343.2832

Forecast Accuracy of: SP500
mape   :  0.0373
me     :  -22.469
mae    :  169.9125
mpe    :  -0.0034
rmse   :  199.7012

Forecast Accuracy of: NASDAQ
mape   :  0.0552
me     :  -283.0841
mae    :  812.3546
mpe    :  -0.0158
rmse   :  1021.2106


# **VARMA with AUTO_ARIMA MODEL**

### **1. Select the order of VARMA with AUTO_ARIMA model**

In [47]:
pq = []
for name, column in data_differenced[['BTC High', 'SP500 High', 'NASDAQ High']].iteritems():
    print(f'Searching order of p and q for : {name}')
    stepwise_model = auto_arima(data_differenced[name],start_p=1, start_q=1,max_p=7, max_q=7, seasonal=False,
        trace=True,error_action='ignore',suppress_warnings=True, stepwise=True,maxiter=1000)
    parameter = stepwise_model.get_params().get('order')
    print(f'optimal order for:{name} is: {parameter} \n\n')
    pq.append(stepwise_model.get_params().get('order'))

Searching order of p and q for : BTC High
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=28102.727, Time=0.10 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=28124.629, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=28100.736, Time=0.07 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=28101.557, Time=0.08 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=28102.716, Time=0.09 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=28092.749, Time=0.46 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=28090.938, Time=0.46 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=28102.545, Time=0.10 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=28082.726, Time=0.50 sec
 ARIMA(4,0,0)(0,0,0)[0]             : AIC=28081.672, Time=0.15 sec
 ARIMA(5,0,0)(0,0,0)[0]             : AIC=28082.766, Time=0.19 sec
 ARIMA(5,0,1)(0,0,0)[0]             : AIC=28084.709, Time=0.64 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : AIC=28082.426, Time=0.33 sec

Best model:  ARIMA(4,0,0)(0,0,0)[0]        

In [48]:
pq

[(4, 0, 0), (0, 0, 2), (2, 0, 0)]

In [49]:
df_results_VARMA_2 = pd.DataFrame(columns=['p', 'q','BTC High2','SP500 High2','NASDAQ High2'])


In [50]:
for i in pq:
    if i[0]== 0 and i[2] ==0:
        pass
    else:
        print(f' Running for {i}')
        model = VARMAX(data_differenced[['BTC High', 'SP500 High','NASDAQ High']], order=(i[0],i[2]))
        model_fit=model.fit(disp=False)
        result = model_fit.forecast(steps = len(data_test))
        inv_res = inverse_diff(data[['BTC High', 'SP500 High','NASDAQ High']], result)
        btcrmse = np.sqrt(metrics.mean_squared_error(data_test['BTC High'], inv_res['BTC High_1st_inv_diff']))
        sprmse = np.sqrt(metrics.mean_squared_error(data_test['SP500 High'], inv_res['SP500 High_1st_inv_diff']))
        NASrmse = np.sqrt(metrics.mean_squared_error(data_test['NASDAQ High'], inv_res['NASDAQ High_1st_inv_diff']))
        df_results_VARMA_2 = df_results_VARMA_2.append({'p': i[0], 'q': i[2], 'BTC High2':btcrmse,
                                                        'SP500 High2':sprmse,
                                                        'NASDAQ High2':NASrmse }, ignore_index=True)

 Running for (4, 0, 0)
 Running for (0, 0, 2)
 Running for (2, 0, 0)


In [51]:
df_results_VARMA_2.sort_values(by=['BTC High2', 'SP500 High2', 'NASDAQ High2'])


,p,q,BTC High2,SP500 High2,NASDAQ High2
0,4.0,0.0,4330.822787,200.103125,1021.069192
2,2.0,0.0,4366.731629,199.779628,1021.153799
1,0.0,2.0,4376.910514,199.555248,1018.083838


In [52]:
best_values_VAR_2 = df_results_VARMA_2.sort_values(by=['BTC High2', 'SP500 High2', 'NASDAQ High2']).head(1)
best_values_VAR_2

,p,q,BTC High2,SP500 High2,NASDAQ High2
0,4.0,0.0,4330.822787,200.103125,1021.069192


the best order is (2,0,0)

In [53]:
p_value_VARMA_2 = best_values_VAR_2['p'].iloc[0]
q_value_VARMA_2 = best_values_VAR_2['q'].iloc[0] 

print("p_value_VARMA_2: ", p_value_VARMA_2)
print("q_value_VARMA_2: ", q_value_VARMA_2)

p_value_VARMA_2:  4.0
q_value_VARMA_2:  0.0


### **2. Model Forecasting**

In [54]:
model = VARMAX(data_differenced[['BTC High', 'SP500 High', 'NASDAQ High']], 
               order=(int(p_value_VARMA_2),int(q_value_VARMA_2))).fit(disp=False)
result = model.forecast(steps = len(data_test))

In [55]:
result2 = inverse_diff(data[['BTC High', 'SP500 High', 'NASDAQ High']],result)
result2.head()

,BTC High,SP500 High,NASDAQ High,BTC High_1st_inv_diff,SP500 High_1st_inv_diff,NASDAQ High_1st_inv_diff
2021-12-19,188.503611,-9.049417,-35.282397,42384.249705,4456.350583,13863.997876
2021-12-20,1.476059,-6.356919,-12.282313,42385.725764,4449.993664,13851.715563
2021-12-21,-178.774146,-0.187321,11.295716,42206.951618,4449.806343,13863.011278
2021-12-22,-126.150233,0.527535,5.246456,42080.801385,4450.333878,13868.257735
2021-12-23,6.769714,1.649295,7.215152,42087.571099,4451.983173,13875.472886


### **3. Visualize the forecast with actual values of VARMA with auto_ARIMA model:**

In [56]:
fig = go.Figure()

fig2 = go.Figure()

fig.add_trace(go.Scatter(x=data_test.index, y=data_test['BTC High'],
                    mode='lines',
                    name='BTC High',
                    line=dict(color='#a03fba')))

fig2.add_trace(go.Scatter(x=data_test.index, y=data_test['SP500 High'],
                    mode='lines', name='SP500 High',
                    line=dict(color='#a03fba')))
fig.add_trace(go.Scatter(x=data_test.index, y=result2['BTC High_1st_inv_diff'],
                    mode='lines',
                    name='BTC High forecast',
                    line=dict(color='#2eb0c7')))


fig2.add_trace(go.Scatter(x=data_test.index, y=result2['SP500 High_1st_inv_diff'],
                    mode='lines', name='SP500 High forecast',
                    line=dict(color='#2eb0c7')))
for plot in [fig,fig2]:
  plot.update_layout(font_color="#d7dbf5",
                  paper_bgcolor="#020938",  
                  plot_bgcolor="#020938",title="Forcast VS Actual")
  plot.update_layout(
      xaxis=dict(
          showline=True,
          showgrid=False,
          showticklabels=True,
          ),
      )
  plot.show()

### **4. Forcast EVALUATION**

In [57]:
from statsmodels.tsa.stattools import acf
def adjust(val, length= 6): return str(val).ljust(length)
def forecast_accuracy(forecast, actual):
    m=np.abs(forecast - actual)/np.abs(actual)
    mape = np.mean(m)  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE

    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse})

print('Forecast Accuracy of: BTC HIGH')
accuracy_prod = forecast_accuracy(result2['BTC High_1st_inv_diff'].values, data_test['BTC High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))


print('\nForecast Accuracy of: SP500')
accuracy_prod = forecast_accuracy(result2['SP500 High_1st_inv_diff'].values, data_test['SP500 High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

print('\nForecast Accuracy of: NASDAQ')
accuracy_prod = forecast_accuracy(result2['NASDAQ High_1st_inv_diff'].values, data_test['NASDAQ High'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,4))

Forecast Accuracy of: BTC HIGH
mape   :  0.0815
me     :  261.774
mae    :  3504.8353
mpe    :  0.0149
rmse   :  4330.8228

Forecast Accuracy of: SP500
mape   :  0.0373
me     :  -23.7674
mae    :  170.1463
mpe    :  -0.0036
rmse   :  200.1031

Forecast Accuracy of: NASDAQ
mape   :  0.0553
me     :  -278.7229
mae    :  813.0777
mpe    :  -0.0155
rmse   :  1021.0692


# **Conclusion**

After studying the causality beteween the three time series, we found out that both S&P500 and NASDAQ series are causing the evolution of Bitcoin.
Then we built VAR,VARMA and VARMA with AUTO-ARIMA and compared performances.
We conclude that the best model is VARMA with auto-ARIMA since it has the least bitcoin price forcasting error.